<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train8_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:

train8 = pd.read_csv(r'/content/train_data8.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [3]:

train8['Target'] = train8['Target'].replace(['Others'],'Invalid')


In [4]:
train8['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [5]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train8_Xtfidf = tfidf_vect.fit_transform(train8['Sentence'])


In [7]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [8]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [9]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [10]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [11]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [12]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train8_Xtfidf,train8['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6858407079646017
                  precision    recall  f1-score   support

        Analysis       0.52      0.40      0.45       127
      Conclusion       0.72      0.51      0.60        45
           Facts       0.76      0.92      0.83       486
         Invalid       0.52      0.48      0.50       130
           Issue       0.55      0.47      0.51        49
Rule/Law/Holding       0.67      0.21      0.32        67

        accuracy                           0.69       904
       macro avg       0.62      0.50      0.53       904
    weighted avg       0.67      0.69      0.66       904



In [14]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [15]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9921777141262098, 0.9902467222700297, 0.9926235077359205, 0.9955427715764353, 0.999897227088931, 0.9981722363955972, 0.9968409057751727, 0.9976122811596042, 0.9954168473125556, 0.9981814030471378, 0.9889958407399435, 0.9944762130189062, 0.99821853514544, 0.9982093516888328, 0.9992914857376507, 0.99198984186402, 0.997131023251033, 0.9952576214384999, 0.9857115799092934, 0.9970244015727429, 0.9992942272275592, 0.9997388820028407, 0.9845841120983011, 0.984556652235597, 0.9924014895618573, 0.9926600566490014, 0.9939659950754156, 0.9994731944502614, 0.9952575246962928, 0.9802431578288131, 0.9997735844093227, 0.9906551393942713, 0.9821698242056018, 0.9881502577191934, 0.9886706027844978, 0.9938145155564262, 0.9929171808173967, 0.9994930214330561, 0.9995303826192711, 0.9976973232048736, 0.9974068296583887]
[0, 1, 2, 4, 5, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 31

In [16]:
unlabel_1=unlabel_1.loc[[0, 1, 2, 4, 5, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 31, 35, 37, 39, 45, 48, 49, 51, 55, 56, 60, 62, 70, 73, 76, 77, 79, 84, 87, 89, 93, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts


In [17]:
frame_1 = [train8,unlabel_1]
train8_1 = pd.concat(frame_1)
len(train8_1)

3517

In [18]:
x_train_1 = tfidf_vect.transform(train8_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train8_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6590909090909091
                  precision    recall  f1-score   support

        Analysis       0.44      0.42      0.43        98
      Conclusion       0.70      0.50      0.58        38
           Facts       0.75      0.89      0.81       360
         Invalid       0.50      0.46      0.48       111
           Issue       0.62      0.49      0.55        37
Rule/Law/Holding       0.58      0.23      0.33        60

        accuracy                           0.66       704
       macro avg       0.60      0.50      0.53       704
    weighted avg       0.64      0.66      0.64       704



In [19]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9931548280370296, 0.9990147697019661, 0.9899366327172798, 0.9931564059237513, 0.9987166669200175, 0.9992546337159, 0.9999497835316231, 0.9865780267593741, 0.9819831646970916, 0.9909592943726673, 0.9876405756495695, 0.9932463839543006, 0.9877958501646492, 0.9989377757422984, 0.9929769837619669, 0.9962418679146064, 0.9943290026795294, 0.9980361235906832, 0.9999394551417713, 0.9989439514723624, 0.9998246374795947, 0.9934701464199347, 0.9994526687006421, 0.981933493329047, 0.993813199957141, 0.9992631259239642, 0.9922189021670652, 0.9991966521378567, 0.9997026857895546, 0.9984516143769749, 0.998134876185756, 0.9876476884880497, 0.990168645799355, 0.995213488866087, 0.9988199775524448, 0.9981970300370976, 0.9963907321093225, 0.9994489849373454, 0.9977455748740739, 0.9918301163469494, 0.9931965317360247, 0.994017236919761]
[0, 2, 3, 4, 7, 8, 12, 13, 16, 17, 20, 21

In [20]:
unlabel_2 = unlabel_2.loc[[0, 2, 3, 4, 7, 8, 12, 13, 16, 17, 20, 21, 23, 29, 31, 32, 34, 36, 37, 39, 42, 44, 49, 52, 55, 59, 60, 63, 64, 69, 70, 71, 75, 77, 79, 83, 85, 88, 90, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [21]:
frame_2 = [train8_1,unlabel_2]
train8_2 = pd.concat(frame_2)
len(train8_2)

3559

In [22]:
x_train_2 = tfidf_vect.transform(train8_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train8_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6566077003121749
                  precision    recall  f1-score   support

        Analysis       0.49      0.41      0.44       145
      Conclusion       0.70      0.37      0.49        51
           Facts       0.74      0.89      0.81       502
         Invalid       0.45      0.45      0.45       144
           Issue       0.71      0.36      0.48        61
Rule/Law/Holding       0.54      0.33      0.41        58

        accuracy                           0.66       961
       macro avg       0.61      0.47      0.51       961
    weighted avg       0.64      0.66      0.64       961



In [23]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9960180532759737, 0.9883766715136975, 0.9972625482668245, 0.9936216157384429, 0.9993875988479831, 0.9866529670184537, 0.9995099638238241, 0.9806156133529802, 0.9810083582922087, 0.998370950058874, 0.988554755620919, 0.9966562985306935, 0.9994779216673388, 0.9820941016615016, 0.9956117578945236, 0.9942365085809918, 0.9986223233801512, 0.9892496262372696, 0.9980741155068871, 0.9984693374143948, 0.9982115118177071, 0.9985876704316433, 0.9970910430116922, 0.9821823212836795, 0.9969990655851052, 0.9941848038889248, 0.9881507733604719, 0.9932005230183997, 0.9996382807620751, 0.9966861185748268, 0.9884246182180068, 0.9994020827213494, 0.9987816542049425, 0.9996595266420114, 0.9945933440554114, 0.9814739199974511, 0.9885954433996489, 0.9988118348255136, 0.9989599895798663, 0.9936409947501361]
[0, 2, 3, 7, 8, 11, 12, 13, 19, 20, 22, 26, 27, 29, 30, 35, 36, 41, 42, 43, 46, 

In [24]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 7, 8, 11, 12, 13, 19, 20, 22, 26, 27, 29, 30, 35, 36, 41, 42, 43, 46, 49, 51, 52, 54, 56, 61, 63, 65, 66, 67, 68, 70, 72, 75, 78, 79, 89, 96, 97],:]
frame_3 = [train8_2,unlabel_3]
train8_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train8_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train8_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6708333333333333
                  precision    recall  f1-score   support

        Analysis       0.49      0.43      0.46       101
      Conclusion       0.75      0.57      0.65        37
           Facts       0.75      0.89      0.81       375
         Invalid       0.50      0.45      0.47       121
           Issue       0.66      0.45      0.54        42
Rule/Law/Holding       0.56      0.32      0.41        44

        accuracy                           0.67       720
       macro avg       0.62      0.52      0.55       720
    weighted avg       0.65      0.67      0.66       720



In [25]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 4, 2, 2, 2, 0, 2, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 3, 2, 3, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2]
[0.9960721335556951, 0.9991036499872135, 0.9992909969518354, 0.99103684189478, 0.9996973211798849, 0.9826242128892437, 0.9979968091843757, 0.9994690834561404, 0.9971409777106716, 0.9903245566839034, 0.9990142539557868, 0.9969471707979087, 0.9987836769093819, 0.9984477458001592, 0.9997161478568685, 0.9989928528725146, 0.9996546292455863, 0.9937295577920312, 0.9976336810367464, 0.9927028270945218, 0.9981828474561966, 0.9886996656322419, 0.9940855408644613, 0.993371359065627, 0.9985056475163167, 0.9956516242841698, 0.999141855243601, 0.9993190412473228, 0.9862649821474894, 0.999972338823325, 0.9851565856848257, 0.9901062574690687, 0.9924900678006515, 0.9975754256297018, 0.9960169281544947, 0.992649315804476]
[2, 4, 7, 8, 10, 12, 15, 18, 22, 25, 27, 28, 29, 32, 34, 37, 40, 42, 46, 50, 56, 58, 64, 66, 67, 69, 70, 71, 72, 75, 79, 82, 85, 87, 97, 99]
36
36
36


In [26]:
unlabel_4=unlabel_4.loc[[2, 4, 7, 8, 10, 12, 15, 18, 22, 25, 27, 28, 29, 32, 34, 37, 40, 42, 46, 50, 56, 58, 64, 66, 67, 69, 70, 71, 72, 75, 79, 82, 85, 87, 97, 99],:]

In [27]:
frame_4 = [train8_3,unlabel_4]
train8_4 = pd.concat(frame_4)
len(train8_4)
x_train_4 = tfidf_vect.transform(train8_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train8_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6883910386965377
                  precision    recall  f1-score   support

        Analysis       0.52      0.44      0.48       145
      Conclusion       0.67      0.47      0.55        59
           Facts       0.79      0.90      0.84       509
         Invalid       0.50      0.51      0.51       148
           Issue       0.55      0.50      0.53        52
Rule/Law/Holding       0.63      0.35      0.45        69

        accuracy                           0.69       982
       macro avg       0.61      0.53      0.56       982
    weighted avg       0.67      0.69      0.67       982



In [28]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0]
[0.9991305815786626, 0.9882495655938177, 0.9947941311348767, 0.9848428873599796, 0.9931050877133307, 0.9983982396595759, 0.9974718374980179, 0.9975214330128944, 0.9995148601930374, 0.983647671805044, 0.9946447940829738, 0.9997987112587617, 0.9942663194811451, 0.9984495746038619, 0.9997066893670357, 0.9810443209140033, 0.9976054445370708, 0.9888387381418857, 0.9896969626987886, 0.9995214678544518, 0.9949456874285281, 0.9980893512966553, 0.9944546394450837, 0.9952160019311668, 0.9811867783535609, 0.9956801168373243, 0.9909092691441125, 0.9998966078426879, 0.9992844913666322, 0.9895207663725721, 0.9931421576308548, 0.9992388544890426, 0.9981804799691463, 0.9932282167547067, 0.9910254533359899, 0.993354776872682, 0.9960091369302277, 0.9901614488065339]
[3, 7, 8, 10, 12, 19, 22, 23, 26, 27, 32, 36, 46, 52, 53, 59, 60, 62, 63, 65, 66, 73, 75, 76, 77, 78, 81, 82, 86, 92, 93, 94, 

In [29]:
unlabel_5 = unlabel_5.loc[[3, 7, 8, 10, 12, 19, 22, 23, 26, 27, 32, 36, 46, 52, 53, 59, 60, 62, 63, 65, 66, 73, 75, 76, 77, 78, 81, 82, 86, 92, 93, 94, 95, 97, 100, 103, 104, 110],:]

In [30]:
frame_5 = [train8_4,unlabel_5]
train8_5 = pd.concat(frame_5)
len(train8_5)
x_train_5 = tfidf_vect.transform(train8_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train8_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6643145161290323
                  precision    recall  f1-score   support

        Analysis       0.50      0.41      0.45       138
      Conclusion       0.59      0.50      0.54        52
           Facts       0.78      0.87      0.82       528
         Invalid       0.45      0.48      0.47       155
           Issue       0.57      0.51      0.54        53
Rule/Law/Holding       0.42      0.21      0.28        66

        accuracy                           0.66       992
       macro avg       0.55      0.50      0.52       992
    weighted avg       0.65      0.66      0.65       992



In [31]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.690978886756238
